# Bayesian Normal Density

In [8]:
# Add beer to the PYTHONPATH
import sys
sys.path.append('../')

import beer
import numpy as np

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

Loading BokehJS ...

In [3]:
def plot_normal(fig, mean, cov, n_sigma=2, alpha=1., color='blue'):
    'Plot a Normal density'
    # Eigenvalue decomposition of the covariance matrix.
    evals, evecs = np.linalg.eigh(cov)
    
    sign = 1 if cov[1, 0] == 0 else np.sign(cov[1, 0])
    # Angle of the rotation.
    angle =  - np.arccos(sign * abs(evecs[0, 0]))
   
    fig.ellipse(x=mean[0], y=mean[1], 
                width=n_sigma * 2* np.sqrt(evals[0]), 
                height=n_sigma * 2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
    fig.ellipse(x=mean[0], y=mean[1], 
                width=2 * np.sqrt(evals[0]), 
                height=2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)

Generate some normally distributed data:

In [4]:
mean = np.array([-1.5, 4])
cov = np.array([
    [5, -1],
    [-1, .5]
])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plot_normal(fig, mean, cov, alpha=0.1)

show(fig)

In [10]:
normal_diag = beer.NormalDiagonalCovariance.create(dim=2)

# Fit the model to the data.
acc_stats = normal_diag.accumulate_stats(data)
normal_diag.natural_grad_update(acc_stats, 1., 1.)

In [11]:
model = beer.models.NormalDiagonalCovariance.create(dim=2)

fig = figure(
    title='Initial model',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plot_normal(fig, mean, cov, alpha=0.1)

plot_normal(fig, normal_diag.mean, normal_diag.cov, alpha=.1, color='red')

show(fig)

In [ ]:
np.cov(data.T)